In [1]:
print("OK")

OK


In [1]:
%pwd

'c:\\Users\\em\\Desktop\\PFA\\Customer_Support_LLMOps\\notebook'

In [2]:
import os 
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\em\\Desktop\\PFA\\Customer_Support_LLMOps'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf_files("data")

In [7]:
extracted_data

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-08-13T21:14:05+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-08-13T21:14:05+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'data\\CODNetwork_Complete_Updated.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content="CODNetwork\nWe Enable Everyone to Start Grow and Scale Ecommerce Business in\nthe Middle East, Africa and Beyond.\nSourcing Fulfillment Call-Center Warehousing Shipping Remittance\n Join us today!\nYou can join us today by choosing the model that suits you. We offer both Seller and Affiliate\nmodels.\nCODNetwork Seller :\nStart selling your own products and goods in multiple countries in MENA region by taking\nadvantage of our services. Transform Your Cash on Delivery Business with COD Network Services.\nFrom sourcing to delivery, we manage your business needs securely, effic

In [8]:
len(extracted_data)

6

In [9]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [10]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [11]:
minimal_docs

[Document(metadata={'source': 'data\\CODNetwork_Complete_Updated.pdf'}, page_content="CODNetwork\nWe Enable Everyone to Start Grow and Scale Ecommerce Business in\nthe Middle East, Africa and Beyond.\nSourcing Fulfillment Call-Center Warehousing Shipping Remittance\n Join us today!\nYou can join us today by choosing the model that suits you. We offer both Seller and Affiliate\nmodels.\nCODNetwork Seller :\nStart selling your own products and goods in multiple countries in MENA region by taking\nadvantage of our services. Transform Your Cash on Delivery Business with COD Network Services.\nFrom sourcing to delivery, we manage your business needs securely, efficiently, and globally.\nOur Solutions\nEnabling online sellers with efficient, quality-focused procurement to delivery solutions, freeing you\nto innovate and market.\nAccount Manager\nYour Ultimate Partner in Business Growth. With an Account Manager at COD Network, reach your\ngoals effortlessly. We enhance your sales and delivery

In [12]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [13]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 22


In [14]:
texts_chunk

[Document(metadata={'source': 'data\\CODNetwork_Complete_Updated.pdf'}, page_content='CODNetwork\nWe Enable Everyone to Start Grow and Scale Ecommerce Business in\nthe Middle East, Africa and Beyond.\nSourcing Fulfillment Call-Center Warehousing Shipping Remittance\n Join us today!\nYou can join us today by choosing the model that suits you. We offer both Seller and Affiliate\nmodels.\nCODNetwork Seller :\nStart selling your own products and goods in multiple countries in MENA region by taking\nadvantage of our services. Transform Your Cash on Delivery Business with COD Network Services.'),
 Document(metadata={'source': 'data\\CODNetwork_Complete_Updated.pdf'}, page_content='From sourcing to delivery, we manage your business needs securely, efficiently, and globally.\nOur Solutions\nEnabling online sellers with efficient, quality-focused procurement to delivery solutions, freeing you\nto innovate and market.\nAccount Manager\nYour Ultimate Partner in Business Growth. With an Account Ma

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\em\AppData\Local\Temp\ipykernel_21304\30168171.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\em\Desktop\PFA\Customer_Support_LLMOps\llmops_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
vector = embedding.embed_query("Hello world")
vector

[-0.03447727486491203,
 0.03102317824959755,
 0.006734970025718212,
 0.026108985766768456,
 -0.03936202451586723,
 -0.16030244529247284,
 0.06692401319742203,
 -0.006441489793360233,
 -0.0474504791200161,
 0.014758856035768986,
 0.07087527960538864,
 0.05552763119339943,
 0.019193334504961967,
 -0.026251312345266342,
 -0.01010954286903143,
 -0.02694045566022396,
 0.022307461127638817,
 -0.022226648405194283,
 -0.14969263970851898,
 -0.017493007704615593,
 0.00767625542357564,
 0.05435224249958992,
 0.0032543970737606287,
 0.031725890934467316,
 -0.0846213847398758,
 -0.02940601296722889,
 0.05159561336040497,
 0.04812406003475189,
 -0.0033148222137242556,
 -0.058279167860746384,
 0.04196927323937416,
 0.022210685536265373,
 0.1281888335943222,
 -0.022338971495628357,
 -0.011656315997242928,
 0.06292839348316193,
 -0.032876335084438324,
 -0.09122604131698608,
 -0.031175347045063972,
 0.0526994913816452,
 0.04703482985496521,
 -0.08420311659574509,
 -0.030056199058890343,
 -0.02074483036

In [18]:
print( "Vector length:", len(vector))

Vector length: 384


In [19]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [20]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [21]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [22]:
pc

In [24]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [25]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
) 

# Add more data to the existing Pinecone index

In [27]:
dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"} 
)

In [28]:
docsearch.add_documents(documents=[dswith])

['7e5df468-3c21-483d-b2b5-ec4a89889a9e']

In [29]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [30]:
retrieved_docs = retriever.invoke("What is CODNetwork?")
retrieved_docs

[Document(id='9f42c050-5de2-4918-b6c7-1d8126ad5724', metadata={'source': 'data\\CODNetwork_Complete_Updated.pdf'}, page_content='CODNetwork\nWe Enable Everyone to Start Grow and Scale Ecommerce Business in\nthe Middle East, Africa and Beyond.\nSourcing Fulfillment Call-Center Warehousing Shipping Remittance\n Join us today!\nYou can join us today by choosing the model that suits you. We offer both Seller and Affiliate\nmodels.\nCODNetwork Seller :\nStart selling your own products and goods in multiple countries in MENA region by taking\nadvantage of our services. Transform Your Cash on Delivery Business with COD Network Services.'),
 Document(id='a512fa45-889c-44bc-9c5f-ea9d0d74a7df', metadata={'source': 'data\\CODNetwork_Info.pdf'}, page_content='CODNetwork We Enable Everyone to Start Grow and Scale Ecommerce Business in the Middle\nEast, Africa and Beyond.\nSourcing Fulfillment Call-Center Warehousing Shipping Remittance\nJoin us today! You can join us today by choosing the model tha

In [31]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o-mini")

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [34]:
system_prompt = (
    "You are CODNetwork’s official customer support assistant.\n"
    "Provide clear, concise, and polite answers to customer questions.\n"
    "Use ONLY the provided context from CODNetwork’s official documentation, FAQs, and product/service descriptions.\n"
    "If the answer is not in the context, politely say you don’t have that information and suggest contacting human support.\n"
    "Always maintain a professional and friendly tone consistent with CODNetwork’s brand.\n"
    "Prioritize accuracy over creativity.\n"
    "Never make up information, policies, or offers.\n"
    "Use simple language customers can understand.\n"
    "If a question is unclear, ask for clarification before answering.\n"
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [35]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [36]:
response = rag_chain.invoke({"input": "what is CODNetwork?"})
print(response["answer"])

CODNetwork is a digital platform that enables individuals and businesses to start, grow, and scale their eCommerce operations in the Middle East, Africa, and beyond. It offers services such as sourcing, fulfillment, call-center support, warehousing, shipping, and remittance. CODNetwork provides two models for joining: the Seller model, where you can sell your own products, and the Affiliate model, where you can sell products without needing capital to purchase inventory.


In [37]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

I'm sorry, but I don't have information on acne. I recommend contacting human support or checking a reliable medical resource for more details.


In [38]:
response = rag_chain.invoke({"input": "what is the Treatment of Acne?"})
print(response["answer"])

I'm sorry, but I don't have information on the treatment of acne. I recommend contacting a healthcare professional for advice on that topic.


In [39]:
response = rag_chain.invoke({"input": "what dswithbappy?"})
print(response["answer"])

I'm sorry, but I don't have information about "dswithbappy." It seems to be outside the context of CODNetwork’s official documentation. I suggest reaching out to our human support for further assistance.
